In [2]:
!pip install azure-ai-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 71.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.9/404.9 kB 23.4 MB/s eta 0:00:00
  Attempting uninstall: azure-storage-blob
    Found existing installation: azure-storage-blob 12.13.0
    Uninstalling azure-storage-blob-12.13.0:
      Successfully uninstalled azure-storage-blob-12.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-mlflow 1.56.0 requires azure-storag

In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

In [20]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)


Found the config file in: /config.json


In [84]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

train_data = Data(
    path=f'./data/train.csv',
    type=AssetTypes.URI_FILE,
    description="Training data asset",
    name="training-data"
)

test_data = Data(
    path=f'./data/test.csv',
    type=AssetTypes.URI_FILE,
    description="Test data asset",
    name="test-data"
)

data_description = Data(
    path=f'./data/data_description.txt',
    type=AssetTypes.URI_FILE,
    description="data description file",
    name="data-desc"
)

data_directory = Data(path = f'./data/', 
type = AssetTypes.URI_FOLDER, 
description="data directory",
name='data-dir')

ml_client.data.create_or_update(train_data)
ml_client.data.create_or_update(test_data)
ml_client.data.create_or_update(data_description)
ml_client.data.create_or_update(data_directory)

Uploading train.csv (< 1 MB): 100%|██████████| 461k/461k [00:00<00:00, 15.1MB/s]


Uploading test.csv (< 1 MB): 100%|██████████| 451k/451k [00:00<00:00, 8.24MB/s]


Uploading data_description.txt (< 1 MB): 100%|██████████| 14.4k/14.4k [00:00<00:00, 1.26MB/s]


Uploading data (0.93 MBs): 100%|██████████| 926841/926841 [00:00<00:00, 10821325.34it/s]




Data({'path': 'azureml://subscriptions/ad5c3a0e-2e80-43f8-a0e0-2cb6463d9e1e/resourcegroups/rg-house-prices-e2e/workspaces/mlw-house-prices-e2e-zaq12wsx/datastores/workspaceblobstore/paths/LocalUpload/354d05cda122399f3a749ed479c47948/data/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'data-dir', 'description': 'data directory', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/ad5c3a0e-2e80-43f8-a0e0-2cb6463d9e1e/resourceGroups/rg-house-prices-e2e/providers/Microsoft.MachineLearningServices/workspaces/mlw-house-prices-e2e-zaq12wsx/data/data-dir/versions/6', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci-zaq12wsx/code/Users/jacekgrela/house-prices-azure-ml/house-prices-e2e-v2.1', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f2b454e576

In [85]:
data = ml_client.data.list()
for datum in data:
    print(datum.name)

training-data
test-data
data-desc
data-dir


In [86]:
import pandas as pd

credential = DefaultAzureCredential()
credential.get_token("https://management.azure.com/.default")
ml_client = MLClient.from_config(credential=credential)
data_asset = ml_client.data.get("training-data",version=6)

pd.read_csv(data_asset.path).head()

Found the config file in: /config.json


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [87]:
credential = DefaultAzureCredential()
credential.get_token("https://management.azure.com/.default")
ml_client = MLClient.from_config(credential=credential)
data_asset = ml_client.data.get("test-data", version=6)

pd.read_csv(data_asset.path).head()

Found the config file in: /config.json


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [88]:
import os

credential = DefaultAzureCredential()
credential.get_token("https://management.azure.com/.default")
ml_client = MLClient.from_config(credential=credential)
data_asset = ml_client.data.get("data-desc", version=6)

# txt file must be downloaded to be read (???)
import azure.ai.ml._artifacts._artifact_utilities as artifact_utils
artifact_utils.download_artifact_from_aml_uri(data_asset.path, 
destination ='.', 
datastore_operation=ml_client.datastores)

with open('./data_description.txt','r') as fp:
    for line in fp.readlines():
        print(line)

os.remove('./data_description.txt')

Found the config file in: /config.json


CAT NOMINAL

MSSubClass: Identifies the type of dwelling involved in the sale.



        20	1-STORY 1946 & NEWER ALL STYLES

        30	1-STORY 1945 & OLDER

        40	1-STORY W/FINISHED ATTIC ALL AGES

        45	1-1/2 STORY - UNFINISHED ALL AGES

        50	1-1/2 STORY FINISHED ALL AGES

        60	2-STORY 1946 & NEWER

        70	2-STORY 1945 & OLDER

        75	2-1/2 STORY ALL AGES

        80	SPLIT OR MULTI-LEVEL

        85	SPLIT FOYER

        90	DUPLEX - ALL STYLES AND AGES

       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER

       150	1-1/2 STORY PUD - ALL AGES

       160	2-STORY PUD - 1946 & NEWER

       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER

       190	2 FAMILY CONVERSION - ALL STYLES AND AGES



CAT NOMINAL

MSZoning: Identifies the general zoning classification of the sale.

		

       A	Agriculture

       C	Commercial

       FV	Floating Village Residential

       I	Industrial

       RH	Residential High Density

       RL	Residential Low Densit

In [89]:
# test

from azure.ai.ml.entities import ManagedIdentityConfiguration
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
credential.get_token("https://management.azure.com/.default")

ml_client = MLClient.from_config(credential=credential)

data_asset = ml_client.data.get("data-desc", version="6")

identity = ManagedIdentityConfiguration()

# to successfully create a job, customize the parameters below based on your workspace resources
job = command(
        command='head ${{inputs.data}}',
        inputs={
          "data": Input(path=data_asset.id,
              type=AssetTypes.URI_FILE,
              mode=InputOutputModes.RO_MOUNT
              )
          },
        environment="azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
        # identity = identity,
        compute = "aml-cluster",
        display_name = 'job-test', 
        experiment_name = 'job-test-exp'
      )
returned_job = ml_client.jobs.create_or_update(job)

Found the config file in: /config.json
